## Code for gesture tracking analysis of Zic2 mice.
This notebook uses .csv with labels extracted from deeplabcut for zic2 mice analysis.

The code usees the session_location parameter to search for the session that the user wants to 
analyse. those sessions are indicated in session.txt.

Overall the code takes the raw matrix of position of each subject and clasiffies 3 types of behaviours:
- Approach
- Contact
- Exploration

It computes a series of variables in the raw_matrix such as:
- behaviour: __str__ behaviour type.
- head_position: 'head_x','head_y': As the midpoint between both ears
- vectors: 
        - x/y_axis_vector: nose head vector
        - x/y crikcet_vector: crikcet head vector
Angles from head to cricket: Angles are computed in the range [-180,180] degrees. When the cricket is facing directly the mouse, the angle is 0.

        - angle_head_cricket: angle in radians
        - deg_head_cricket: angle in degrees

Other variables:
- Distances in px and cm
    - distance_px/cm: Distance made by the animal in each time bin. Using the center body label as reference
    - distance_head_cricket: Distance from head midpoint to cricket
    - diff_head_cricket: Distance change between head and cricket in each time bin.
- temporal variables:
    - time: Time of each time bin un secons.
    - diff_time: time diffence across eah time bin 
    - speed_cm_s: speed of the mouse on each time bin, taking into account the body postion label
    - to_cricket_cm_s: Speed related to diff_head_cricket
    - gradient: positive or negative gradient [+1,-1] indicating cricket head approach[-1], cricket head distance increase [-1]
    
- beahviour_counter : add_behaviour_counter(): To asign an increasing interger value each time there is a behviour change.  

The code loops through each session and computes:

- behaviour_matrix_df: Agreggated behaviour matrix filtered by the behaviour_thr. 
                        behaviour_thr determines how long mut a behaviour last to be considered one of the three behaviours.
- raw_matrix_df: Raw matrix dataframe with computed variables
- subject_list: a list of objects, where each object is an independent subject.


Window of analysis: Data is filetered by the ini_time and end_time determined in capture_df.xlsx.

## Behaviour data preprocessing

In [2]:
# sys.path.append('../../')
import pandas as pd
from pathlib import Path
import numpy as np
from os import listdir
from os.path import isfile, join,isdir
import commons as cm
import matplotlib.pyplot as plt

### Parameters

In [3]:
# Location of deep lab cut processed data
session_location = Path(r"dlc_preprocessed_data\videos") 

data_path=Path(r"analysed/") #folder whre csv will be saved
model_name='DLC_resnet50_zic2Nov29shuffle1_850000' #deep lab cut model used for analysis

arena=(50,50) # arena size in cm
pixels_cm=4.11
# approach='to_cricket_cm_s<7 & gradient<0 &\
#           distance_head_cricket_cm>4 & \
#           speed_cm_s>10 & \
#           deg_head_cricket>-45 & deg_head_cricket<45'

to_hz=16 # downsample all videos to 16 hz
approach='to_cricket_cm_s<-7 & gradient<0 &\
          distance_head_cricket_cm>4.5 & \
          speed_cm_s>10'
contact='distance_head_cricket_cm<=4.5'
behaviour_thr=0.1#minimum time to be considered an exploration or an approach
likelihood_thr=0.26
behaviour_color={'exploration':'green','approach':'blue','contact':'red'}

Get the list of directories we want to analyse and copy paste the information in sessions.tx

In [4]:
sessions=[]

onlyfiles =[file.replace('.avi','') for file 
                        in [f for f in listdir(session_location) if isfile(join(session_location, f))]
                        if '.avi' in file]
            
for dir in onlyfiles:
 print (dir,flush=True)

f_26_day_2_trial_1_20230607_101443
f_26_day_2_trial_2_20230607_104447
f_26_day_2_trial_3_20230607_110101
f_28_day_2_trial_1_20230607_103612
f_28_day_2_trial_2_20230607_105431
f_28_day_2_trial_3_20230607_111434
f_479_day_2_trial_1_20230804_121649
f_479_day_2_trial_2_20230804_123322
f_479_day_2_trial_3_20230804_125105
f_623_day_2_trial_1_20230804_120916
f_623_day_2_trial_2_20230804_122540
f_623_day_2_trial_3_20230804_124239
m_483_day_2_trial_1_20230802_155743
m_483_day_2_trial_2_20230802_162029
m_483_day_2_trial_3_20230802_163726
m_61_day_2_trial_1_20230613_123611
m_61_day_2_trial_2_20230613_130118
m_61_day_2_trial_3_20230613_132512
m_629_day_2_trial_1_20230802_155252
m_629_day_2_trial_2_20230802_161542
m_629_day_2_trial_3_20230802_163229
m_62_day_2_trial_1_20230613_124154
m_62_day_2_trial_2_20230613_130818
m_62_day_2_trial_3_20230613_133037
m_630_day_2_trial_1_20230802_151715
m_630_day_2_trial_2_20230802_152945
m_630_day_2_trial_3_20230802_154015
m_63_day_2_trial_1_20230613_125045
m_63_

### Extract the path of the desired experiments


In [5]:
### Do the analysis
with open('session.txt','r') as f:
    for session in f:
            sessions.append(session.replace('\n',''))
f.close()

session_loc_list=[session_location.joinpath(session+model_name+'_filtered') for session in sessions ]
session_loc_list


[WindowsPath('Z:/lab/escritorio/projects/gesture_tracking/zic2_publication_code/zic2/dlc_preprocessed_data/videos/f_26_day_2_trial_1_20230607_101443DLC_resnet50_zic2Nov29shuffle1_850000_filtered'),
 WindowsPath('Z:/lab/escritorio/projects/gesture_tracking/zic2_publication_code/zic2/dlc_preprocessed_data/videos/f_26_day_2_trial_2_20230607_104447DLC_resnet50_zic2Nov29shuffle1_850000_filtered'),
 WindowsPath('Z:/lab/escritorio/projects/gesture_tracking/zic2_publication_code/zic2/dlc_preprocessed_data/videos/f_26_day_2_trial_3_20230607_110101DLC_resnet50_zic2Nov29shuffle1_850000_filtered'),
 WindowsPath('Z:/lab/escritorio/projects/gesture_tracking/zic2_publication_code/zic2/dlc_preprocessed_data/videos/f_28_day_2_trial_1_20230607_103612DLC_resnet50_zic2Nov29shuffle1_850000_filtered'),
 WindowsPath('Z:/lab/escritorio/projects/gesture_tracking/zic2_publication_code/zic2/dlc_preprocessed_data/videos/f_28_day_2_trial_2_20230607_105431DLC_resnet50_zic2Nov29shuffle1_850000_filtered'),
 WindowsPa

### Load window of analysis dataframe
This dataframe contains the timestamps to cut the regions of interest for each session

In [6]:
capture_df=pd.read_excel('capture_df.xlsx')
capture_df=capture_df.astype({'capture_ini':'Int64',
                   'capture_end':'Int64',
                   'capture_final':'Int64'},errors='ignore')
capture_df['behaviour_thr']=behaviour_thr
display(capture_df.head(),'length: '+ str(len(capture_df)))


,session,sex,mouse_num,day,trial,date,capture_ini,capture_end,capture_final,frame_rate,behaviour_thr
0,f_26_day_1_trial_1_20230606_160648,f,26,1,1,2023-06-06 16:06:48,190,13633,13779,40,0.1
1,f_26_day_1_trial_2_20230606_164028,f,26,1,2,2023-06-06 16:40:28,16,1960,2027,16,0.1
2,f_26_day_1_trial_3_20230606_170647,f,26,1,3,2023-06-06 17:06:47,40,2898,3098,40,0.1
3,f_26_day_2_trial_1_20230607_101443,f,26,2,1,2023-06-07 10:14:43,144,3056,3056,16,0.1
4,f_26_day_2_trial_2_20230607_104447,f,26,2,2,2023-06-07 10:44:47,90,1383,1383,16,0.1


'length: 193'

### Load phenotype dataframe
Dataframe with the phenotype of each number each mouse


In [7]:
phenotype_df=pd.read_excel('phenotype_df.xlsx')
phenotype_df.head()

,mouse_num,phenotype,sex
0,26,wt,f
1,27,tg,f
2,28,wt,f
3,479,tg,f
4,623,wt,f


### Lests merge de datafarames

In [8]:
capture_df=capture_df.merge(phenotype_df.loc[:,['mouse_num','phenotype']],on='mouse_num',how='inner')
display(capture_df.head())

,session,sex,mouse_num,day,trial,date,capture_ini,capture_end,capture_final,frame_rate,behaviour_thr,phenotype
0,f_26_day_1_trial_1_20230606_160648,f,26,1,1,2023-06-06 16:06:48,190,13633,13779,40,0.1,wt
1,f_26_day_1_trial_2_20230606_164028,f,26,1,2,2023-06-06 16:40:28,16,1960,2027,16,0.1,wt
2,f_26_day_1_trial_3_20230606_170647,f,26,1,3,2023-06-06 17:06:47,40,2898,3098,40,0.1,wt
3,f_26_day_2_trial_1_20230607_101443,f,26,2,1,2023-06-07 10:14:43,144,3056,3056,16,0.1,wt
4,f_26_day_2_trial_2_20230607_104447,f,26,2,2,2023-06-07 10:44:47,90,1383,1383,16,0.1,wt


In [9]:
(capture_df.query('frame_rate>20 ')
 .groupby(['phenotype','mouse_num'])
 .agg(count=('trial',len)))

count
phenotype mouse_num       
tg        27             3
          479            6
          487            4
          62             3
          63             1
          801            5
          88             2
          91             3
          93             4
          94             3
          98             5
wt        26             3
          28             5
          483            5
          61             3
          623            3
          629            6
          630            5
          64             4
          87             1
          90             1
          92             6
          c              8

### Start analysis

In [10]:
# subject_list=[]
behaviour_matrix_list=[]
raw_data_list=[]

for session, recording_location in zip(sessions,session_loc_list):
    data_paths=cm.extract_data_paths(recording_location=recording_location,
                          subject=session)
    subject=cm.Gesture(data_paths=data_paths,name=session,pixels_cm=pixels_cm)
    print(f'session:{session}')
    #get window of analysis for each session
    session_index=capture_df['session'].str.contains(session)
    capture_ini=capture_df.loc[session_index,'capture_ini'].values[0] # intial frame for analysis
    capture_end=capture_df.loc[session_index,'capture_end'].values[0] # final frame for analysis
    subject.video_hz=capture_df.loc[session_index,'frame_rate'].values[0]
    subject.to_hz=to_hz
    

    display(f'Initial window frame: {capture_ini}')
    display(f'Final window frame: {capture_end}')

    subject.frames_to_matrix()
    subject.check_data_length()

    # # Cut data by window of interest
    subject.frames=subject.frames[capture_ini:capture_end]
    subject.time=subject.time[capture_ini:capture_end]
    subject.data=subject.data.iloc[capture_ini:capture_end].reset_index(drop=True)
    subject.check_data_length() # check that video data and data matrix are of equal length
    
    subject.downsampling()

    subject.check_data_length() # check that video data and data matrix are of equal length

    subject.add_head_midpoint()
    subject.get_vectors()
    subject.orientation_head_cricket()
    subject.add_time()
    subject.add_distances()
    subject.add_temporal_data()
    subject.categorize_behaviours(approach=approach,contact=contact)

    # clean low likelihood data
    columns=subject.data.columns[subject.data.columns.str.contains('likelihood')].drop('Tail_likelihood')
    likelihood_index=(subject.data.loc[:,columns]<likelihood_thr).any(axis=1)
    subject.data.loc[likelihood_index,'behaviour']=pd.NA
    subject.add_behaviour_counter_refactor()

    # clean first row
    subject.data=subject.data.iloc[1:] # drop first row avoid border effects np.diff diferences in speeds and so on
    subject.frames=subject.frames[1:] # drop alose first frame from video

    # aggregate data for beahviour matrix
    subject.compute_behaviour_matrix(behaviour_thr=behaviour_thr)

    #clean low threshold data:
    # change to nan behaviours under a therdhold. When gruoping by nan values  pandas excludes them
    # cool way to filter out data in the posterior analysis
    
    threshold_index=(subject.behaviour_matrix['diff_time']<behaviour_thr) & (subject.behaviour_matrix['behaviour']!='contact')
    beh_counter_index=subject.behaviour_matrix.loc[threshold_index,'beh_counter']
    beh_counter_index=subject.data['beh_counter'].isin(beh_counter_index)
    subject.data.loc[beh_counter_index,['behaviour']]=pd.NA    
    
    #compute final behaviour matrix by frouping filtered data and reccaltuling behaviour counter
    subject.add_behaviour_counter_refactor()
    subject.compute_behaviour_matrix(behaviour_thr=behaviour_thr)

    # # #Save intial and final imaes for comprobotaions
    # plt.imsave(f'analysed/images/first_frame_{session}'+'.jpg',subject.frames[0],format='jpg')
    # plt.imsave(f'analysed/images/last_frame_{session}'+'.jpg',subject.frames[-1],format='jpg')

    subject.save_processed_video(video_hz=to_hz,
                                 video_name=f'analysed/videos/{session}_thr_{behaviour_thr}'+'.avi')
    
    # #add a session column for the final matrix
    subject.behaviour_matrix.insert(loc=0,column='session',value=session) 
    subject.behaviour_matrix=capture_df.merge(subject.behaviour_matrix,on='session',how='inner')

    subject.data.insert(loc=0,column='session',value=session)
    subject.data=capture_df.merge(subject.data,on='session',how='inner')

    for col in ['session', 'behaviour']:
         subject.behaviour_matrix[col]=subject.behaviour_matrix[col].astype('category')
         subject.data[col]=subject.data[col].astype('category')

    # subject_list.append(subject)
    behaviour_matrix_list.append(subject.behaviour_matrix)
    raw_data_list.append(subject.data)

behaviour_matrix_df=pd.concat(behaviour_matrix_list) # aggregated behaviour data filtered by behaviour threshold
raw_matrix_df=pd.concat(raw_data_list) # raw data with added columns and classified behhaviour counts

behaviour_matrix_df.to_excel(f'analysed/tables/behaviour_matrix_df_thr_{behaviour_thr}.xlsx',index=False)
raw_matrix_df.to_excel(f'analysed/tables/raw_matrix_df_thr_{behaviour_thr}.xlsx',index=False)
    

session:f_26_day_2_trial_1_20230607_101443


'Initial window frame: 144'

'Final window frame: 3056'

maintaing frame rate
Categorize behaviours in approaches, contact and exploration
append behaviours to main dataframe


z:\lab\escritorio\projects\gesture_tracking\zic2_publication_code\zic2\zic2\commons.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approach_df['behaviour']='approach'
z:\lab\escritorio\projects\gesture_tracking\zic2_publication_code\zic2\zic2\commons.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contact_df['behaviour']='contact'
z:\lab\escritorio\projects\gesture_tracking\zic2_publication_code\zic2\zic2\commons.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

...

'Initial window frame: 18'

'Final window frame: 282'

maintaing frame rate
Categorize behaviours in approaches, contact and exploration
append behaviours to main dataframe


z:\lab\escritorio\projects\gesture_tracking\zic2_publication_code\zic2\zic2\commons.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approach_df['behaviour']='approach'
z:\lab\escritorio\projects\gesture_tracking\zic2_publication_code\zic2\zic2\commons.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contact_df['behaviour']='contact'
z:\lab\escritorio\projects\gesture_tracking\zic2_publication_code\zic2\zic2\commons.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

Video saved successfully!


In [14]:
subject.frames

array([], dtype=float64)